In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary


ModuleNotFoundError: No module named 'torchinfo'

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.block(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=[64, 128, 256]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Downsampling path
        for feature in features:
            self.downs.append(ConvBlock(in_channels, feature))
            in_channels = feature

        # Bottleneck
        self.bottleneck = ConvBlock(features[-1], features[-1]*2)

        # Upsampling path
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(ConvBlock(feature*2, feature))

        # Final output
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for i in range(0, len(self.ups), 2):
            x = self.ups[i](x)
            skip_connection = skip_connections[i//2]

            # If needed, pad to match size (due to pooling/upsampling mismatch)
            if x.shape != skip_connection.shape:
                x = F.pad(x, [0, skip_connection.shape[3] - x.shape[3],
                              0, skip_connection.shape[2] - x.shape[2]])

            x = torch.cat((skip_connection, x), dim=1)
            x = self.ups[i+1](x)

        return self.final_conv(x)

In [ ]:
model = UNet(in_channels=3, out_channels=3)
x = torch.randn(1, 3, 256, 256)
y = model(x)
print(y.shape)  # Expected: (1, 3, 256, 256)


torch.Size([1, 3, 256, 256])


In [ ]:
summary(model, input_size=(1, 3, 256, 256))